In [1]:
from longeval.spark import get_spark
from longeval.collection import ParquetCollection
from pyspark.sql import functions as F

spark = get_spark()
root = "../../data/longeval/parquet/"
collection = ParquetCollection(spark, f"{root}/train/2023_01/English")

24/12/24 19:29:59 WARN Utils: Your hostname, daphne-major resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/24 19:29:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 19:30:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/24 19:30:00 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
row = collection.queries.sample(False, 0.1, 42).head()
print(row)

Row(qid='q0123805', query='Veal filet mignon')


In [3]:
collection.qrels.printSchema()
df = collection.qrels.where(F.col("qid") == row.qid)
df.count(), df.distinct().count()
relevant_set = {row.docid for row in df.collect() if row.rel > 0}
relevant_set

root
 |-- qid: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)



{'doc012303512545',
 'doc012304816815',
 'doc012305205106',
 'doc012307703830',
 'doc012308900581',
 'doc012312602366'}

In [4]:
from opensearchpy import OpenSearch

client = OpenSearch("http://localhost:9200")
client.indices.get(index="train-english-2023_01")

{'train-english-2023_01': {'aliases': {},
  'mappings': {'properties': {'contents': {'type': 'text'},
    'docid': {'type': 'keyword'}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '1',
    'provided_name': 'train-english-2023_01',
    'creation_date': '1735090123643',
    'number_of_replicas': '0',
    'uuid': 'lLIZckjbQJmCW95zprKRyQ',
    'version': {'created': '136397827'}}}}}

In [5]:
# get queries and calculate ndcg for the first 1000 documents
from sklearn.metrics import ndcg_score

query = row.query
results = client.search(
    index="train-english-2023_01",
    body={"query": {"match": {"contents": query}}},
    size=1000,
)
docids = [hit["_source"]["docid"] for hit in results["hits"]["hits"]]
print(docids[:10])
true_relevance = [1 if docid in relevant_set else 0 for docid in docids]
predicted_relevance = [1] * len(docids)
ndcg_score([true_relevance], [predicted_relevance]), sum(true_relevance)

['doc012303612112', 'doc012308900581', 'doc012311208151', 'doc012303512545', 'doc012303115394', 'doc012310507994', 'doc012307102434', 'doc012311313775', 'doc012312300386', 'doc012300503573']


(np.float64(0.2234867697570255), 6)